# **Demo on building data prep pipeline for model fine tuning**

<a href="https://colab.research.google.com/github/santoshborse/pydatanyc2024/blob/main/fine-tune-code.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This demo notebook shows how to use [data-prep-kit](https://github.com/IBM/data-prep-kit) to build a data preparation pipeline that can be used for fine tuning or extended pre-training. We will discuss the various data preparation steps to process raw data (code repositories), tokenise it that can then be fine tuned using any popular code models. We will also discuss a novel recipe for semantic ordering of files in a repository which has shown to enhance model training. Please see our [paper](https://arxiv.org/abs/2407.13739) here for more details. For this demo, we will use the [codeparrot/github-code](https://huggingface.co/datasets/codeparrot/github-code) dataset hosted on Hugging Face datasets.



## Setup

Install data-prep-toolkit and datasets library. This notebook requires atleast 8 cpus.
To run on google colab, it is recommended to change the runtime to TPUs to get the required number of cpus.


In [ ]:
%%capture logpip --no-stderr
!pip3 install 'data-prep-toolkit[ray]==0.2.2.dev1'
!pip3 install 'data-prep-toolkit-transforms[ray,all]==0.2.2.dev1'
!pip install datasets
!pip install pandas

In [ ]:
!wget -O 'utils.py'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/utils.py'


--2024-11-05 16:32:53--  https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1856 (1.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   1.81K  --.-KB/s    in 0s      

2024-11-05 16:32:53 (41.0 MB/s) - ‘utils.py’ saved [1856/1856]



We use parallel processing capability using Ray, so that beyond the demo, a user can also use this for actual production runs on larger datasets, with minor code changes. Please read [here](https://github.com/IBM/data-prep-kit?tab=readme-ov-file#-about-) on various features of data-prep-kit that includes flexibility of compute to run from laptop to cluster.  There are three parameters, that the user can change, as per usecase:

`runtime_num_worker`: number of parallel workers to be used

`num_cpus`: number of cpus to be used per worker

`run_locally: True` start a ray cluster for parallel computation


In [ ]:
from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.utils import ParamsUtils
import sys
import json
import pandas as pd
#Default parameters for computation
worker_options = {"num_cpus": 0.3}
common_config_params = {
  "run_locally": True,
  "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
  "runtime_num_workers": 2,
}




We will do all the processing in `sample_data` folder. This concludes our setup section.

In [ ]:
!rm -rf sample_data
!mkdir -p sample_data
!mkdir -p sample_data/hf_2_parquet

## Data Preparation Steps

We now discuss the various data preparation steps to transform the raw data to a tokenised format post cleaning and transforming the data. We use the [parquet data format](https://parquet.apache.org/) for all our operations. This helps to efficiently scale the data for actual production runs, beyond the demo.

1. HuggingFace2Parquet: Read the dataset from HF and convert into parquet format.
2. Exact Deduplication: Remove exact duplicates.
3. Fuzzy Deduplication: Remove near duplicates.
4. Programming Lang Selection: Select the programming languages to be used for the analysis.
5. Code Quality Annotations: Annotate whether a given code file is of high quality or not using various rules.
6. Filtering: Filter dataset to retain only programming language of interest.
7. Semantic Ordering: Organise code files by their semantic dependencies.  
8. Tokenization: Tokenise the data for model fine tuning.

The data processing pipeline is organised such that the output of the previous transform is used as input to the next one. Refer to the papers [here](https://arxiv.org/pdf/2405.04324) and [here](https://arxiv.org/abs/2407.13739) for complete details for each of the above steps.

## 1. Huggingface datasets to Parquet

This is the first component of this pipeline. It ingests a dataset `codeparrot/github-code` from huggingface and converts it into
parquet files for consumption by the next steps in this data processing pipeline.

For this demo we are trying to process a few records. The following fields can be updated in case you want to use more data.

_total_files_ = 20 <br/>
_rows_per_file_ = 20

The output of this stage of the pipeline would be written to `sample_data/hf_2_parquet`.

In [ ]:
import os
import pyarrow as pa
import pyarrow.parquet as pq

from datasets import load_dataset

import uuid
from data_processing.utils import TransformUtils
from collections import defaultdict

DATASET_NAME='codeparrot/github-code'

ds = load_dataset(DATASET_NAME,
                  streaming=True,
                  split="train",
                  trust_remote_code=True)

def row_mapper(row):
    return {
            'ext': TransformUtils.get_file_extension(row['path'])[1],
            'document_id': str(uuid.uuid4())
            }

parquet_data_output = "sample_data/hf_2_parquet"
!rm -rf "sample_data/"
!mkdir -p "sample_data/hf_2_parquet"

## Converts a subset of a Hugging Face dataset to a Parquet file, optionally mapping and renaming columns.
def hf_dataset_to_parquet(ds, skip, nrows, file_name, mapper=None, renamed_columns=[]):
    dst_ = ds.skip(skip).take(nrows)

    data_dict = defaultdict(list)

    dst = dst_.map(mapper)

    for data in dst:
        for k, v in data.items():
            data_dict[k].append(v)

    for old, new in renamed_columns:
        data_dict[new] = data_dict[old]
        del data_dict[old]

    table = pa.Table.from_pydict(data_dict)
    pq.write_table(table, file_name)


# Create parquet files

total_files = 20
rows_per_file = 20
for num in range(total_files):
    file_name = os.path.join(
        f"{parquet_data_output}",
        f"data_{num}.parquet"
    )
    print (f"Writing {file_name}")
    hf_dataset_to_parquet(ds,
                          1 * rows_per_file,
                          rows_per_file,
                          file_name=file_name,
                          mapper=row_mapper,
                          renamed_columns=[("code", "contents"),
                                           ("path", "title")])

Writing sample_data/hf_2_parquet/data_0.parquet
Writing sample_data/hf_2_parquet/data_1.parquet
Writing sample_data/hf_2_parquet/data_2.parquet
Writing sample_data/hf_2_parquet/data_3.parquet
Writing sample_data/hf_2_parquet/data_4.parquet
Writing sample_data/hf_2_parquet/data_5.parquet
Writing sample_data/hf_2_parquet/data_6.parquet
Writing sample_data/hf_2_parquet/data_7.parquet
Writing sample_data/hf_2_parquet/data_8.parquet
Writing sample_data/hf_2_parquet/data_9.parquet
Writing sample_data/hf_2_parquet/data_10.parquet
Writing sample_data/hf_2_parquet/data_11.parquet
Writing sample_data/hf_2_parquet/data_12.parquet
Writing sample_data/hf_2_parquet/data_13.parquet
Writing sample_data/hf_2_parquet/data_14.parquet
Writing sample_data/hf_2_parquet/data_15.parquet
Writing sample_data/hf_2_parquet/data_16.parquet
Writing sample_data/hf_2_parquet/data_17.parquet
Writing sample_data/hf_2_parquet/data_18.parquet
Writing sample_data/hf_2_parquet/data_19.parquet


In [ ]:
from utils import read_parquet_files_as_df
print(f"{parquet_data_output = }")
output_df = read_parquet_files_as_df(parquet_data_output)
print ("Output dimensions (rows x columns)= ", output_df.shape)
output_df.head(400)

parquet_data_output = 'sample_data/hf_2_parquet'
Output dimensions (rows x columns)=  (400, 8)


,repo_name,language,license,size,ext,document_id,contents,title
0,decred/dcrd,GO,isc,10267,.go,351be2b7-ba63-44f1-ac58-0a970a4c3cf9,// Copyright (c) 2021 The Decred developers\n/...,blockchain/indexers/indexsubscriber.go
1,Brawl345/WiiDataDownloader-Module,Batchfile,isc,521,.bat,a63b3b96-6d3d-4a06-9383-7197f8d85fa9,@echo off\nCLS\n%header%\necho.\nif not exist ...,HackMii Installer herunterladen.bat
2,decred/decrediton,Markdown,isc,1305,.md,5d1ed08a-ed57-4a93-af1c-8bdbb0cff6fd,# Limit rozbieżności\n\n**Ostrzeżenie! Ustawie...,app/i18n/docs/pl/InfoModals/GapLimit.md
3,k0gaMSX/scc,C,isc,1127,.h,ba35df7a-b1b9-47c1-9fde-9dc9de94f122,"enum asmop {\n\tASNOP = 0,\n\tASSTB,\n\tASSTH,...",cc2/target/qbe/arch.h
4,Dirbaio/btcd,GO,isc,28011,.go,2e35ddaa-e6e7-4ae4-bade-e5838edc7809,// Copyright (c) 2013-2015 The btcsuite develo...,wire/msgblock_test.go
...,...,...,...,...,...,...,...,...
395,nickleefly/gulp-boilerplate,Markdown,isc,152,.md,7d51023f-05f5-4d27-98f7-c18f556aa34f,# gulp boilerplate\n\nrun\n`npm start`\n\nthen...,README.md
396,nicholaides/Methanol-Web-Crawler,C,isc,6361,.c,d1ba416c-423b-47af-a79f-e8881e7a5d76,/*-\n * builtin.c\n * This file is part of lib...,src/libmetha/builtin.c
397,dennis95/dennix,C,isc,1293,.c,13ddcfbb-4686-4e90-aeba-ece167acae09,"/* Copyright (c) 2019, 2022 Dennis Wölfing\n *...",libc/src/stdio/__file_write.c
398,la5942/argentum-invoice,PHP,isc,1554,.php,c6b0e1d9-ecc2-4e58-8b43-9e2d7420ae4f,<?php\n/**\n * Time Controller\n *\n * @packag...,application/controllers/admin/time.php


## 2. Exact deduplication

This step will find exact duplicates in the 'content' column and remove them. This is done by computing SHA256 hash on the code files and remove records having identical hashes.

The transform specific params for exact deduplication are: <br/>
 _ededup_hash_cpu_ -  Number of cpus per worker <br/>
 _ededup_num_hashes_ - Number of workers used to store hashes <br/>
 _ededup_doc_column_ - Name of column which has to be checked for deduplication <br/>


In [ ]:
import os
import sys
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration

input_folder = parquet_data_output # Output of previous stage is used as input.
output_folder = "sample_data/ededup_out"

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

ededup_params = {
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}

params = common_config_params | ededup_params
sys.argv = ParamsUtils.dict_to_req(d=params)
ededup_launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
ededup_launcher.launch()

17:40:30 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
INFO:ededup_transform_base:exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
17:40:30 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
17:40:30 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
17:40:30 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
17:40:30 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
17:40:30 INFO - job details {'job category': 'preproc

0

In [ ]:
import json
import pprint
def read_metadata(path):
    with open(path, 'r') as file:
        metadata = json.load(file)
        pprint.pp(metadata)
read_metadata(f"{output_folder}/metadata.json")

{'pipeline': 'pipeline_id',
 'job details': {'job category': 'preprocessing',
                 'job name': 'ededup',
                 'job type': 'ray',
                 'job id': 'job_id',
                 'start_time': '2024-11-05 17:40:33',
                 'end_time': '2024-11-05 17:40:36',
                 'status': 'success'},
 'code': None,
 'job_input_params': {'doc_column': 'contents',
                      'doc_id_column': 'document_id',
                      'use_snapshot': False,
                      'snapshot_directory': None,
                      'hash_cpu': 0.5,
                      'num_hashes': 2,
                      'checkpointing': False,
                      'max_files': -1,
                      'random_samples': -1,
                      'files_to_use': ['.parquet'],
                      'number of workers': 2,
                      'worker options': {'num_cpus': 0.3, 'max_restarts': -1},
                      'actor creation delay': 0},
 'execution_stats':

In [ ]:


from utils import read_parquet_files_as_df
print(output_folder)
output_df = read_parquet_files_as_df(output_folder)
print ("Output dimensions (rows x columns)= ", output_df.shape)
import glob
parquet_files = glob.glob(f'{output_folder}/*.parquet')
parquet_files
dfs = [pd.read_parquet (f) for f in parquet_files]

for df in dfs:
  print(df.shape)


sample_data/ededup_out
Output dimensions (rows x columns)=  (20, 9)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(20, 9)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)
(0, 8)


## 3. Fuzzy Deduplication

This step will find near duplicates and remove them. The code is broken into two code cells, one for adding document ids to the parquet file and then running fuzzy dedup. Document id addition is a prerequisite for fuzzy dedup.

We first add the document ids as an additional column to the parquet files. <br/>
_doc_column_ - specifies name of the column containing the document (required for ID generation) <br/>
_hash_column_ - specifies name of the column created to hold the string document id, if None, id is not generated <br/>
_int_id_column_ - specifies name of the column created to hold the integer document id, if None, id is not generated <br/>
At least one of hash_column or int_id_column must be specified.



In [ ]:
input_folder = "sample_data/ededup_out"
output_folder = "sample_data/docid_out"


from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_id_params = {
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}

params = doc_id_params | common_config_params
sys.argv = ParamsUtils.dict_to_req(d=params)
launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())
launcher.launch()

17:49:31 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column', 'start_id': 0}
INFO:doc_id_transform_base:Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column', 'start_id': 0}
17:49:31 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
17:49:31 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
17:49:31 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
17:49:31 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
17:49:31 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
INFO:data_

0

In [ ]:
from utils import read_parquet_files_as_df
input_folder = "sample_data/ededup_out"
output_folder = "sample_data/docid_out"

input_df = read_parquet_files_as_df(input_folder)
output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"New columns Added: {set(output_df.columns)-set(input_df.columns)}")


output_df.head(5)


Input data dimensions (rows x columns)=  (20, 9)
Output data dimensions (rows x columns)=  (20, 11)
New columns Added: {'hash_column', 'int_id_column'}


,repo_name,language,license,size,ext,document_id,contents,title,removed,hash_column,int_id_column
0,decred/dcrd,GO,isc,10267,.go,f6aa9b7a-f9f5-4cf8-90cf-b3b3cab5f875,// Copyright (c) 2021 The Decred developers\n/...,blockchain/indexers/indexsubscriber.go,[],c0474fa8cb7053c9de4d0c9d05642e1fc7013bf7081cd4...,0
1,Brawl345/WiiDataDownloader-Module,Batchfile,isc,521,.bat,05daa6ed-8eb7-4599-be93-4987b9315cb7,@echo off\nCLS\n%header%\necho.\nif not exist ...,HackMii Installer herunterladen.bat,[],c39d53bb1287b3abaea70fd53d02dd66435e887672be45...,1
2,decred/decrediton,Markdown,isc,1305,.md,8895fbdb-d9e3-4782-ae19-bddf460c50c6,# Limit rozbieżności\n\n**Ostrzeżenie! Ustawie...,app/i18n/docs/pl/InfoModals/GapLimit.md,[],c8b6cffc41424729e77dddf77294ae93427bc5a3fc38de...,2
3,k0gaMSX/scc,C,isc,1127,.h,0725f0c8-b8e4-4542-9a17-bec5a2aff730,"enum asmop {\n\tASNOP = 0,\n\tASSTB,\n\tASSTH,...",cc2/target/qbe/arch.h,[],c16bc266105cfdd27e256d7a6d8edca26e12f786f4d0f0...,3
4,Dirbaio/btcd,GO,isc,28011,.go,bcb2d875-bbac-4c16-ace1-8ed14c1e1dd5,// Copyright (c) 2013-2015 The btcsuite develo...,wire/msgblock_test.go,[],0280cb33cd5e7916d0c44d0319f5956d84cad9867ac22a...,4


In [ ]:
read_metadata(f"{output_folder}/metadata.json")

{'pipeline': 'pipeline_id',
 'job details': {'job category': 'preprocessing',
                 'job name': 'doc_id',
                 'job type': 'ray',
                 'job id': 'job_id',
                 'start_time': '2024-11-05 17:49:35',
                 'end_time': '2024-11-05 17:49:37',
                 'status': 'success'},
 'code': None,
 'job_input_params': {'doc_column': 'contents',
                      'hash_column': 'hash_column',
                      'int_column': 'int_id_column',
                      'start_id': 0,
                      'checkpointing': False,
                      'max_files': -1,
                      'random_samples': -1,
                      'files_to_use': ['.parquet'],
                      'number of workers': 2,
                      'worker options': {'num_cpus': 0.3, 'max_restarts': -1},
                      'actor creation delay': 0},
 'execution_stats': {'cpus': 96,
                     'gpus': 0,
                     'memory': 224.6836

Post adding the document ids, the next step is to run fuzzy deduplication. We apply a two-step method for this: (1) compute MinHashes of all the documents and then utilize Locally Sensitive Hashing (LSH) to group documents based on their MinHash fingerprints, (2) measure Jaccard similarity between each pair of documents
in the same bucket and annotate documents except one as duplicates based on a similarity
threshold.  

Some important transform specific params are: <br/>
_fdedup_doc_column_ - Column to be used for deduplication <br/>
_fdedup_threshold_ - specifies the Jaccard similarity threshold (default is 0.7)

In [ ]:
input_folder = "sample_data/docid_out"
output_folder = "sample_data/fdedup_out"

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus": 0.8}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
fdedup_params = {
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}

params = common_config_params| fdedup_params

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch
fdedup_launcher = RayTransformLauncher(FdedupRayTransformConfiguration())
fdedup_launcher.launch()

17:51:40 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 0.3}}
INFO:fdedup_transform_ray:fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 

0

In [ ]:
read_metadata(f"{output_folder}/metadata.json")

In [ ]:
input_folder = "sample_data/docid_out"
output_folder = "sample_data/fdedup_out"
input_df = read_parquet_files_as_df(input_folder)
output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"New columns Added: {set(output_df.columns)-set(input_df.columns)}")


output_df.head(5)

Input data dimensions (rows x columns)=  (20, 11)
Output data dimensions (rows x columns)=  (20, 11)
New columns Added: set()


,repo_name,language,license,size,ext,document_id,contents,title,removed,int_id_column,hash_column
0,decred/dcrd,GO,isc,10267,.go,f6aa9b7a-f9f5-4cf8-90cf-b3b3cab5f875,// Copyright (c) 2021 The Decred developers\n/...,blockchain/indexers/indexsubscriber.go,[],0,-1
1,Brawl345/WiiDataDownloader-Module,Batchfile,isc,521,.bat,05daa6ed-8eb7-4599-be93-4987b9315cb7,@echo off\nCLS\n%header%\necho.\nif not exist ...,HackMii Installer herunterladen.bat,[],1,-1
2,decred/decrediton,Markdown,isc,1305,.md,8895fbdb-d9e3-4782-ae19-bddf460c50c6,# Limit rozbieżności\n\n**Ostrzeżenie! Ustawie...,app/i18n/docs/pl/InfoModals/GapLimit.md,[],2,-1
3,k0gaMSX/scc,C,isc,1127,.h,0725f0c8-b8e4-4542-9a17-bec5a2aff730,"enum asmop {\n\tASNOP = 0,\n\tASSTB,\n\tASSTH,...",cc2/target/qbe/arch.h,[],3,-1
4,Dirbaio/btcd,GO,isc,28011,.go,bcb2d875-bbac-4c16-ace1-8ed14c1e1dd5,// Copyright (c) 2013-2015 The btcsuite develo...,wire/msgblock_test.go,[],4,-1


## 4. Programming Language Selection

This module helps retain the code files for language of interest which can be specified using selected_languages_file. Post this step, a new column is added, that contains the programming language name. One can use the code in the Filtering step to do analytics on how many files are found for which languages and thereby selectively filter.

The important parameters used by this transform are: <br/>
_lang_allowed_langs_file_key_ - A file with a list of allowed languages. <br/>
_lang_lang_column_key_ - The name of column which has programming language. <br/>
_lang_output_column_key_ - The name of annotation column. <br/>

For this demo, we will use this [file](https://github.com/IBM/data-prep-kit/blob/dev/transforms/code/proglang_select/python/test-data/languages/allowed-code-languages.txt) to specify languages of interest and the module will add a new column called "language_of_interest" which can have two values 0/1. 1 is added for all rows that have code files belonging to programming language specified in the list.

In [ ]:
input_folder = "sample_data/fdedup_out"
output_folder = "sample_data/ps_out"

# download allowed-code-languages.txt
# !wget https://raw.githubusercontent.com/IBM/data-prep-kit/dev/transforms/code/proglang_select/python/test-data/languages/allowed-code-languages.txt

# Create a file with language of interest
! echo -e "JavaScript\nC++\nC\nGO\nJava" >> allowed-code-languages.txt

selected_languages_file = "./allowed-code-languages.txt"

from proglang_select_transform_ray import ProgLangSelectRayConfiguration
from proglang_select_transform import (
    lang_allowed_langs_file_key,
    lang_lang_column_key,
    lang_output_column_key,
)

# create parameters
language_column_name = "language"
annotated_column_name = "language_of_interest"

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

langselect_config = {
    lang_allowed_langs_file_key: selected_languages_file,
    lang_lang_column_key: language_column_name,
    lang_output_column_key: annotated_column_name,
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}

params = common_config_params| langselect_config

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(ProgLangSelectRayConfiguration())
launcher.launch()

17:55:36 INFO - data factory proglang_select_ is using local configuration without input/output path
INFO:data_processing.data_access.data_access_factory_base7c32ee3d-836a-4c4f-82cf-e6a598f4fb66:data factory proglang_select_ is using local configuration without input/output path
17:55:36 INFO - data factory proglang_select_ max_files -1, n_sample -1
INFO:data_processing.data_access.data_access_factory_base7c32ee3d-836a-4c4f-82cf-e6a598f4fb66:data factory proglang_select_ max_files -1, n_sample -1
17:55:36 INFO - data factory proglang_select_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
INFO:data_processing.data_access.data_access_factory_base7c32ee3d-836a-4c4f-82cf-e6a598f4fb66:data factory proglang_select_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
17:55:36 INFO - pipeline id pipeline_id
INFO:data_processi

0

In [ ]:
input_folder = "sample_data/fdedup_out"
output_folder = "sample_data/ps_out"
input_df = read_parquet_files_as_df(input_folder)
output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"New columns Added: {set(output_df.columns)-set(input_df.columns)}")

output_df.head(5)

Input data dimensions (rows x columns)=  (20, 11)
Output data dimensions (rows x columns)=  (20, 12)
New columns Added: {'language_of_interest'}


,repo_name,language,license,size,ext,document_id,contents,title,removed,int_id_column,hash_column,language_of_interest
0,decred/dcrd,GO,isc,10267,.go,f6aa9b7a-f9f5-4cf8-90cf-b3b3cab5f875,// Copyright (c) 2021 The Decred developers\n/...,blockchain/indexers/indexsubscriber.go,[],0,-1,True
1,Brawl345/WiiDataDownloader-Module,Batchfile,isc,521,.bat,05daa6ed-8eb7-4599-be93-4987b9315cb7,@echo off\nCLS\n%header%\necho.\nif not exist ...,HackMii Installer herunterladen.bat,[],1,-1,False
2,decred/decrediton,Markdown,isc,1305,.md,8895fbdb-d9e3-4782-ae19-bddf460c50c6,# Limit rozbieżności\n\n**Ostrzeżenie! Ustawie...,app/i18n/docs/pl/InfoModals/GapLimit.md,[],2,-1,False
3,k0gaMSX/scc,C,isc,1127,.h,0725f0c8-b8e4-4542-9a17-bec5a2aff730,"enum asmop {\n\tASNOP = 0,\n\tASSTB,\n\tASSTH,...",cc2/target/qbe/arch.h,[],3,-1,True
4,Dirbaio/btcd,GO,isc,28011,.go,bcb2d875-bbac-4c16-ace1-8ed14c1e1dd5,// Copyright (c) 2013-2015 The btcsuite develo...,wire/msgblock_test.go,[],4,-1,True


## 5. Code Quality

We experiment with various code quality metrics but finally retain the four code quality metrics used by (Li et al., 2023) to balance the tradeoff between code quality versus data volume.

Quality metrics

'line_mean': Average of the total line lengths.
'line_max':  Maximum line length present .
'total_num_lines': Total number of lines present
'avg_longest_lines': Average of the first n longest lines, where n can be any number you choose.
'alphanum_frac':  Calculates average of alpha numeric with respect to total data
'char_token_ratio': Computes character/token ratio of the file with tokenizer
'autogenerated': Check if file is autogenerated by looking for keywords in the first few lines of the file.
'config_or_test':  Check if file is a configuration file or a unit test
'has_no_keywords': Check if a python file has none of the keywords - for funcion, class, for loop, while loop.
'has_few_assignments': Check if file uses symbol '=' less than 'minimum' times
'is_xml': Check if input data is xml content
'is_html': Check if input data is HTML files based on displayed text VS code ratio

In [ ]:
input_folder = "sample_data/ps_out"
output_folder = "sample_data/cq_out"

from code_quality_transform_ray import CodeQualityRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
# ??


language_column_name = "language"
params = {
    "cq_contents_column_name": "contents",
    "cq_language_column_name": language_column_name,
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}

params = common_config_params| params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(CodeQualityRayTransformConfiguration())
# launch
launcher.launch()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

17:57:59 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
17:57:59 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
17:57:59 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
17:57:59 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
17:57:59 INFO - job details {'job category': 'preprocessing', 'job name': 'code_quality', 'job type': 'ray', 'job id': 'job_id'}
INFO:data_processing_ray.runtime.ray.execution_configuration:job details {'job category': 'preprocessing', 'job name': 'code_quality', 'job type': 'ray', 'job id': 'job_id'}
17:57:59 INFO - data factory data_ is using local data access: input_folder - sample_data/ps_out output_folder - sample_data/cq

0

In [ ]:
read_metadata(f"{output_folder}/metadata.json")

In [ ]:
input_folder = "sample_data/ps_out"
output_folder = "sample_data/cq_out"
input_df = read_parquet_files_as_df(input_folder)
output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"New columns Added: {set(output_df.columns)-set(input_df.columns)}")

output_df.head(5)

Input data dimensions (rows x columns)=  (20, 12)
Output data dimensions (rows x columns)=  (20, 24)
New columns Added: {'autogenerated', 'is_xml', 'avg_longest_lines', 'line_max', 'config_or_test', 'has_no_keywords', 'total_num_lines', 'char_token_ratio', 'is_html', 'line_mean', 'alphanum_frac', 'has_few_assignments'}


,repo_name,language,license,size,ext,document_id,contents,title,removed,int_id_column,...,total_num_lines,avg_longest_lines,alphanum_frac,char_token_ratio,autogenerated,config_or_test,has_no_keywords,has_few_assignments,is_xml,is_html
0,decred/dcrd,GO,isc,10267,.go,f6aa9b7a-f9f5-4cf8-90cf-b3b3cab5f875,// Copyright (c) 2021 The Decred developers\n/...,blockchain/indexers/indexsubscriber.go,[],0,...,390,90.571429,0.698549,3.408699,False,False,False,False,False,False
1,Brawl345/WiiDataDownloader-Module,Batchfile,isc,521,.bat,05daa6ed-8eb7-4599-be93-4987b9315cb7,@echo off\nCLS\n%header%\necho.\nif not exist ...,HackMii Installer herunterladen.bat,[],1,...,11,69.428571,0.738964,2.492823,False,False,False,False,False,False
2,decred/decrediton,Markdown,isc,1305,.md,8895fbdb-d9e3-4782-ae19-bddf460c50c6,# Limit rozbieżności\n\n**Ostrzeżenie! Ustawie...,app/i18n/docs/pl/InfoModals/GapLimit.md,[],2,...,15,170.142857,0.822186,1.743954,False,False,False,False,False,False
3,k0gaMSX/scc,C,isc,1127,.h,0725f0c8-b8e4-4542-9a17-bec5a2aff730,"enum asmop {\n\tASNOP = 0,\n\tASSTB,\n\tASSTH,...",cc2/target/qbe/arch.h,[],3,...,135,10.428571,0.658385,1.506684,False,False,False,True,False,False
4,Dirbaio/btcd,GO,isc,28011,.go,bcb2d875-bbac-4c16-ace1-8ed14c1e1dd5,// Copyright (c) 2013-2015 The btcsuite develo...,wire/msgblock_test.go,[],4,...,791,85.142857,0.642105,2.457968,False,True,False,False,False,False


## 6. Filtering

This step can be used to filter the code files based on our chosen conditions. In this demo example, we have only used one annotation of adding programming language names for each code file. To demonstrate the utility, we will use this module to retain only code files of interest.

In [ ]:
input_folder = "sample_data/cq_out"
output_folder = "sample_data/filter_out"


from filter_transform import (
    filter_columns_to_drop_cli_param,
    filter_criteria_cli_param,
    filter_logical_operator_cli_param,
)
from filter_transform_ray import FilterRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

# This is just an example criteria to filter
filter_criteria = [
    "language_of_interest = 1",
    "total_num_lines > 10 AND total_num_lines < 90"
]
filter_logical_operator = "AND"
filter_columns_to_drop = ["language_of_interest", "hash_column"]

filter_params = {
    filter_criteria_cli_param: filter_criteria,
    filter_columns_to_drop_cli_param: filter_columns_to_drop,
    filter_logical_operator_cli_param: filter_logical_operator,
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}


sys.argv = ParamsUtils.dict_to_req(common_config_params| filter_params)
launcher = RayTransformLauncher(FilterRayTransformConfiguration())
launcher.launch()


17:59:39 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
17:59:39 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
17:59:39 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
17:59:39 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
17:59:39 INFO - job details {'job category': 'preprocessing', 'job name': 'filter', 'job type': 'ray', 'job id': 'job_id'}
INFO:data_processing_ray.runtime.ray.execution_configuration:job details {'job category': 'preprocessing', 'job name': 'filter', 'job type': 'ray', 'job id': 'job_id'}
17:59:39 INFO - data factory data_ is using local data access: input_folder - sample_data/cq_out output_folder - sample_data/filter_out
INF

0

In [ ]:
input_folder = "sample_data/cq_out"
output_folder = "sample_data/filter_out"
input_df = read_parquet_files_as_df(input_folder)
output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"New columns Removed: {set(input_df.columns)-set(output_df.columns)}")

output_df.head(5)

Input data dimensions (rows x columns)=  (20, 24)
Output data dimensions (rows x columns)=  (5, 22)
New columns Removed: {'hash_column', 'language_of_interest'}


,repo_name,language,license,size,ext,document_id,contents,title,removed,int_id_column,...,total_num_lines,avg_longest_lines,alphanum_frac,char_token_ratio,autogenerated,config_or_test,has_no_keywords,has_few_assignments,is_xml,is_html
0,sec51/clamav-yara,GO,isc,436,.go,26fdbd5a-3f45-4f90-ba78-2bfac6e8a0b3,"package main\n\nimport (\n\t""testing""\n)\n\nfu...",hdb_signatures_test.go,[],8,...,19,51.285714,0.740826,2.520231,False,False,False,False,False,False
1,TealCube/strife,Java,isc,2828,.java,79a2a145-5783-43db-9948-b8f282b2afa6,/**\n * The MIT License Copyright (c) 2015 Tea...,src/main/java/land/face/strife/managers/Combat...,[],10,...,77,96.571429,0.720651,3.837178,False,False,False,False,False,False
2,dennis95/dennix,C,isc,1043,.c,4840e9a4-8d3c-4904-9022-e418e47768fa,"/* Copyright (c) 2016, 2021 Dennis Wölfing\n *...",libc/src/stdio/printf.c,[],13,...,29,73.714286,0.730326,3.830882,False,False,False,True,False,False
3,dennis95/dennix,C,isc,1293,.c,3b0ac5e3-5997-4dd6-99a9-ba763b71d22f,"/* Copyright (c) 2019, 2022 Dennis Wölfing\n *...",libc/src/stdio/__file_write.c,[],17,...,37,73.714286,0.682663,3.822485,False,False,False,False,False,False
4,bolav/fusepm,JavaScript,isc,752,.js,0e53f69a-54b3-4280-b1a9-127f38b145db,var fusepm = require('./fusepm');\n\nmodule.ex...,lib/fixunoproj.js,[],19,...,37,40.428571,0.507979,2.387302,False,False,False,False,False,False


## 7. Semantic Ordering of Code Files

In this step, we order the code files such that we pack files from the same repository together, arranging them to prioritize semantic dependencies. We identify these dependencies by analyzing file imports and create a directed acyclic graph, where each file is a node and edges represent API imports between files. After breaking any cycles in the graph, we perform a topological sort to establish an ordering of files based on their semantic dependencies. We then organize the files in a repository by placing documentation and build files first, followed by the ordered set of files with semantic dependencies, and finally the remaining non-connected files. These non-connected files are arranged according to their folder structure, using a depth-first search to traverse the repository. Finally, we determine the dominant programming language of a repository based on file extensions and presence of build files, to organise repo-ordered files by programming languages.


This transform has following parameters:  <br/>
 _repo_lvl_sorting_enabled_ - If True, the repo level output is sorted using _repo_lvl_sorting_algo_ <br/>
 _repo_lvl_sorting_algo_ - Select the sorting algorithm to be used for repo level sorting. Use SORT_SEMANTIC_NORMALISED to organise by semantic dependencies or SORT_BY_PATH to arrange files based on folder structure in a repository.  <br/>
 _repo_lvl_store_backend_dir_ -  Directory to use for local store. Needed only when repo_lvl_store_type=local <br/>
 _repo_lvl_output_by_langs_ - If True, it organises output into folders of programming language. <br/>
 _repo_lvl_combine_rows_ - If True, it combines the contents of repo into a single row. <br/>



In [ ]:
!git --version
# !add-apt-repository ppa:git-core/ppa
# !apt update
# !apt install git
# !git --version

git version 2.47.0


In [ ]:
input_folder = "sample_data/filter_out"
output_folder = "sample_data/rlo_out"

import tempfile
from repo_level_order_transform import RepoLevelOrderRayTransformConfiguration
with tempfile.TemporaryDirectory() as tmpdirname:

    # create parameters
    local_conf = {
        "input_folder": input_folder,
        "output_folder": output_folder,
     }

    worker_options = {"num_cpus": 0.8}
    code_location = {"github": "github", "commit_hash": "12345", "path": "path"}

    repo_level_params = {
        "repo_lvl_sorting_algo": "SORT_SEMANTIC_NORMALISED",
        "repo_lvl_store_type": "local",
        "repo_lvl_store_backend_dir": tmpdirname,
        "repo_lvl_output_by_langs": True,
        "repo_lvl_combine_rows": True,
        "repo_lvl_sorting_enabled": True,
        "data_local_config": ParamsUtils.convert_to_ast(local_conf)
    }


    sys.argv = ParamsUtils.dict_to_req(d= common_config_params| repo_level_params)
    launcher = RayTransformLauncher(RepoLevelOrderRayTransformConfiguration())
    launcher.launch()

18:16:38 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
18:16:38 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
18:16:38 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
18:16:38 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
18:16:38 INFO - job details {'job category': 'preprocessing', 'job name': 'repo_lvl', 'job type': 'ray', 'job id': 'job_id'}
INFO:data_processing_ray.runtime.ray.execution_configuration:job details {'job category': 'preprocessing', 'job name': 'repo_lvl', 'job type': 'ray', 'job id': 'job_id'}
18:16:38 INFO - data factory data_ is using local data access: input_folder - sample_data/filter_out output_folder - sample_data/rlo_ou

Creating Store Params


2024-11-05 18:16:39,882	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(orchestrate pid=130357) 18:16:41 INFO - orchestrator started at 2024-11-05 18:16:41
(orchestrate pid=130357) 18:16:41 INFO - Number of files is 1, source profile {'max_file_size': 0.015949249267578125, 'min_file_size': 0.015949249267578125, 'total_file_size': 0.015949249267578125}
(orchestrate pid=130357) 18:16:41 INFO - Cluster resources: {'cpus': 96, 'gpus': 0, 'memory': 224.72495060041547, 'object_store': 100.30207557603717}
(orchestrate pid=130357) 18:16:41 INFO - Number of workers - 2 with {'num_cpus': 0.3, 'max_restarts': -1} each
(orchestrate pid=130357) 18:16:41 INFO - => get_transform_config started
(orchestrate pid=130357) 18:16:41 INFO - dict_keys(['store_backend_dir', 'store_type', 's3_creds'])
(orchestrate pid=130357) 18:16:41 INFO - <= get_transform_config


(orchestrate pid=130357) Init Store params
(RayTransformFileProcessor pid=135685) Creating local store.


(orchestrate pid=130357) 18:16:43 INFO - Completed 0 files (0.0%)  in 0.0 min. Waiting for completion
(orchestrate pid=130357) 18:16:44 INFO - Completed processing 1 files in 0.005 min
(orchestrate pid=130357) 18:16:44 INFO - done flushing in 0.003 sec
(orchestrate pid=130357) 18:16:44 INFO - Store Backend is None
(orchestrate pid=130357) 18:16:44 INFO - Stage 1 Finished in 0:00:02.382250.
(orchestrate pid=130357) 18:16:45 I - Repo level sorting is enabled. Algo: SORT_SEMANTIC_NORMALISED
(orchestrate pid=130357) 18:16:45 I - normalised semantic sort enabled
(orchestrate pid=130357) 18:16:45 I - Output by language enabled.
(orchestrate pid=130357) 18:16:45 I - Combine rows enabled.
(orchestrate pid=130357) 18:16:47 I - Processing 4 repos with 2 workers
(GroupByRepoActor pid=138254) 18:16:47 I - Write GO/sec51%2Fclamav-yara, tables: 1
(orchestrate pid=130357) 18:16:48 I - Finished the transform in 0:00:06.423414 
18:16:58 INFO - Completed execution in 0.329 min, execution result 0
INFO:d

(orchestrate pid=130357) Creating local store. [repeated 2x across cluster]


In [ ]:
read_metadata(f"{output_folder}/metadata.json")

{'pipeline': 'pipeline_id',
 'job details': {'job category': 'preprocessing',
                 'job name': 'repo_lvl',
                 'job type': 'ray',
                 'job id': 'job_id',
                 'start_time': '2024-11-05 18:16:41',
                 'end_time': '2024-11-05 18:16:48',
                 'status': 'success'},
 'code': None,
 'job_input_params': {'stage_one_only': False,
                      'grouping_column': 'repo_name',
                      'language_column': 'language',
                      'store_type': 'local',
                      'store_backend_dir': '/tmp/tmpt3_7x29l',
                      'store_ray_cpus': 0.5,
                      'store_ray_nworkers': 1,
                      'sorting_enabled': True,
                      'sorting_algo': 'SORT_SEMANTIC_NORMALISED',
                      'output_by_langs': True,
                      'combine_rows': True,
                      'ray_workers': 2,
                      'ray_num_cpus': 0.3,
       

## 8. Tokenization

Next, we tokenize the data to be used for fine tuning.



In [ ]:
input_folder = "sample_data/rlo_out"
output_folder = "sample_data/tokenize_out"

from tokenization_transform_ray import TokenizationRayConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

tf_params= {
    "data_local_config": ParamsUtils.convert_to_ast(local_conf)
}
sys.argv = ParamsUtils.dict_to_req(d=common_config_params| tf_params)
# create launcher
launcher = RayTransformLauncher(TokenizationRayConfiguration())
# Launch the ray actor(s) to process the input
launcher.launch()

18:20:29 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
18:20:29 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
18:20:29 INFO - number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
INFO:data_processing_ray.runtime.ray.execution_configuration:number of workers 2 worker options {'num_cpus': 0.3, 'max_restarts': -1}
18:20:29 INFO - actor creation delay 0
INFO:data_processing_ray.runtime.ray.execution_configuration:actor creation delay 0
18:20:29 INFO - job details {'job category': 'preprocessing', 'job name': 'Tokenization', 'job type': 'ray', 'job id': 'job_id'}
INFO:data_processing_ray.runtime.ray.execution_configuration:job details {'job category': 'preprocessing', 'job name': 'Tokenization', 'job type': 'ray', 'job id': 'job_id'}
18:20:29 INFO - data factory data_ is using local data access: input_folder - sample_data/rlo_out output_folder - sample_data/t

0

In [ ]:
read_metadata(f"{output_folder}/metadata.json")

{'pipeline': 'pipeline_id',
 'job details': {'job category': 'preprocessing',
                 'job name': 'Tokenization',
                 'job type': 'ray',
                 'job id': 'job_id',
                 'start_time': '2024-11-05 18:20:35',
                 'end_time': '2024-11-05 18:20:41',
                 'status': 'success'},
 'code': None,
 'job_input_params': {'tokenizer': 'hf-internal-testing/llama-tokenizer',
                      'tokenizer_args': None,
                      'doc_id_column': 'document_id',
                      'doc_content_column': 'contents',
                      'text_lang': 'en',
                      'chunk_size': 0,
                      'checkpointing': False,
                      'max_files': -1,
                      'random_samples': -1,
                      'files_to_use': ['.parquet'],
                      'number of workers': 2,
                      'worker options': {'num_cpus': 0.3, 'max_restarts': -1},
                      'actor

In [ ]:

output_df = read_parquet_files_as_df(f"{output_folder}/*")
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

Output data dimensions (rows x columns)=  (4, 4)


,tokens,document_id,document_length,token_count
0,"[1, 4949, 14187, 1266, 313, 29883, 29897, 2987...",878b8ba9-b4ed-4f60-8c46-e8763045f130,2334,850
1,"[1, 722, 285, 1509, 3358, 353, 1996, 877, 6904...",ebb689cb-71a4-463d-8757-5c2843b8c673,752,387
2,"[1, 3577, 1667, 13, 13, 5215, 313, 13, 12, 299...",d42f28b8-9bb8-4477-8360-4250a2857160,436,213
3,"[1, 7762, 13, 334, 450, 341, 1806, 19245, 1418...",82be553f-6594-4101-8c34-9c83ccb712ac,2828,894


**The data is now ready for extended pretraining or fine tuning using any open source code models.**